#import library and connect to google drive

In [49]:
import numpy as np
import pandas as pd
import nltk
!pip install hazm
import hazm
import string
import re
from google.colab import drive
from math import log
drive.mount('/content/drive')
from google.colab import output
output.clear()

#load file

In [6]:
temp = pd.read_csv("/content/drive/MyDrive/machine learning/12 - sentiment words/data/sad.csv")
temp = temp.iloc[0 : 500]
df_sad = pd.DataFrame()
df_sad["Text"] = temp["tweet"]
df_sad["emotin"] = [0 for _ in range(df_sad.shape[0])]

temp = pd.read_csv("/content/drive/MyDrive/machine learning/12 - sentiment words/data/joy.csv")
temp = temp.iloc[0 : 500]
df_joy = pd.DataFrame()
df_joy["Text"] = temp["tweet"]
df_joy["emotin"] = [1 for _ in range(df_joy.shape[0])]
df = pd.concat([df_sad, df_joy], axis=0)

In [7]:
#sad
df.head(5)

,Text,emotin
0,خیلی خوشحالم از اینکه تو کشوری به دنیا امدم که...,0
1,رود\nبی محتوای آبی عشق\nکوه\nبی خنده های کبک ج...,0
2,تا زمانی برات دلسوزی میکنن که هرچی اونها میگن ...,0
3,علامه حسن زاده آملی ؛\nدو شهید خیلی از من اشک ...,0
4,خواستم عاشقیم پیش خودم باشد و تو\nگریـه هایم چ...,0


In [9]:
#joy
df.tail(5)

,Text,emotin
495,این #حمایت_حداکثری #مردم از #رئیسی\nمثل نماز ا...,1
496,واقعا اگر مشارکت ۲۵٪ باشه من خودم به محض اینک...,1
497,واسه جشن دلتنگی ما، گل گریه سبد سبد بود\nبا طل...,1
498,خوابم نمیبره، حس اون فردی رو دارم که داره برای...,1
499,هر چه تبر زدن مرا\nزخم نشد جوانه شد\nعزیزان شم...,1


#pre process data

In [10]:
persian_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
punctuations_list = string.punctuation + persian_punctuations
arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
stop_words = hazm.stopwords_list()
lemmatizer = hazm.Lemmatizer()


def _remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text

def _remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

def _remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

def _normalize_persian(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ي", "ی", text)
    text = re.sub("ؤ", "و", text)
    text = re.sub("ئ", "ی", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("ك" ,"ک" , text)
    text = re.sub("[^ابپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی]", " ", text)
    text = re.sub("[^\S\n\t]+", ' ', text)
    return text

def _tokenize(text):
    return text.split()

def _remove_stopwords(words):
    return [word  for word in words if word not in stop_words]

def _lemmatizer(words):
    result = set()
    for token in words:
        result.add(lemmatizer.lemmatize(token))
    return list(result)



df['Text'] = df['Text'].apply(_remove_diacritics)
df['Text'] = df['Text'].apply(_remove_punctuations)
df['Text'] = df['Text'].apply(_remove_repeating_char)
df['Text'] = df['Text'].apply(_normalize_persian)
df['Text'] = df['Text'].apply(_tokenize)
df['Text'] = df['Text'].apply(_remove_stopwords)
df['Text'] = df['Text'].apply(_lemmatizer)

In [11]:
#sad
df.head(5)

,Text,emotin
0,"[امدم, نمیرسم, تلاش, چی, خوشحال, کشور, دنیا]",0
1,"[روز, شیونفومنی, مجنون, اشک, تجسم, باد, ماند#م...",0
2,"[هرچی, میگن, انجام, انتخابات, وگرنه, زمانی, اش...",0
3,"[امل, اشک, روح, میکنیم, اله, حسن, صیاد, مطهر, ...",0
4,"[خادمان, چقدر, خودم, گفت#گو, اشک, مرا, صحن, گر...",0


In [12]:
#joy
df.tail(5)

,Text,emotin
495,"[وفریاد, کاسبانمذاکرهتکراردروغ, جشنپیروزی, انه...",1
496,"[باشه, خودم, محض, نتایج, میگیرم, بشه, بوق, واق...",1
497,"[بد, گل, سبد, دلتنگی, گریه, جشن, ستاره, طلوع, ...",1
498,"[خو, میشه, شور, خفت#خواب, داره, حساب, اتحاد, خ...",1
499,"[تبر, پیروزی, شمیم, عزیز, مرا, جوانه, زدن, میک...",1


#Collect words

In [27]:
words = list()

for Ws in df["Text"]:
  words.extend(Ws)
words = list(set(words))
words[:10]

['پیشتون',
 'حضورتون',
 'پروپاگاندانتخاباتی',
 'مانتالیزم',
 'شفافیتارا',
 'یاس',
 'مسیولینبیلیاقت',
 'میکنه',
 'بازه',
 'حالدلتونچطوره']

#count freqencies

In [42]:
count_freq = pd.DataFrame()
count_freq["words"] = words
count_freq.set_index("words", inplace = True)
count_freq["freq postive"] = [0 for _ in range(len(words))]
count_freq["freq negetive"] = [0 for _ in range(len(words))]

for index in range(df.shape[0]):
  temp_words = df.iloc[index]["Text"]
  for word in temp_words:
    if df.iloc[index]["emotin"] == 1:
      count_freq.loc.__setitem__(([word], "freq postive"), count_freq.loc[word]["freq postive"] + 1)

    if df.iloc[index]["emotin"] == 0:
      count_freq.loc.__setitem__(([word], "freq negetive"), count_freq.loc[word]["freq negetive"] + 1)

count_freq.head(10)

,freq postive,freq negetive
words,,
پیشتون,0,1
حضورتون,1,0
پروپاگاندانتخاباتی,1,0
مانتالیزم,0,1
شفافیتارا,0,1
یاس,1,0
مسیولینبیلیاقت,0,1
میکنه,1,5
بازه,2,1


#$\lambda$ calculation  



In [56]:
n_postive = sum(count_freq["freq postive"])
n_negetive = sum(count_freq["freq negetive"])
v_class = len(words)

def calculate_ratio(freq, target):
  if target == 0:
    return (freq + 1) / (v_class + n_negetive)
  if target == 1:
    return (freq + 1) / (v_class + n_postive)

count_freq["lambda"] = [None for _ in range(count_freq.shape[0])]
for word in words:
  pos = count_freq.loc[word]["freq postive"]
  neg = count_freq.loc[word]["freq negetive"]
  pos_ratio = calculate_ratio(pos, 1)
  neg_ratio = calculate_ratio(neg, 0)

  lambda_cal = log((pos_ratio / neg_ratio))
  count_freq.loc.__setitem__(([word], "lambda"), lambda_cal)

count_freq["label"] = ["postive" if e > 0 else "negetive" for e in count_freq["lambda"]]
count_freq.head(15)


,freq postive,freq negetive,lambda,label
words,,,,
پیشتون,0,1,-0.675799,negetive
حضورتون,1,0,0.710495,postive
پروپاگاندانتخاباتی,1,0,0.710495,postive
مانتالیزم,0,1,-0.675799,negetive
شفافیتارا,0,1,-0.675799,negetive
یاس,1,0,0.710495,postive
مسیولینبیلیاقت,0,1,-0.675799,negetive
میکنه,1,5,-1.081265,negetive
بازه,2,1,0.422813,postive
